# Controller

In [22]:
class remoteController:
  def __init__(item, startFloor1, startFloor2, floor_count):
    item.floor_count = floor_count
    item.elevator1 = mainElevator(startFloor1, 1)
    item.elevator2 = mainElevator(startFloor2, 2)
    item.stack = []
    item.stack_elevator1 = []
    item.stack_elevator2 = []
    item.currentStep = None
    item.nextStep = None
    item.isError = False

  def initStack(item, stack):
    item.stack.extend(stack)

  def findItem(item, command):
    if command[0] < command[1]:
      return "goUp"
    elif command[0] > command[1]:
      return "goDown"

  def checkEmpty(item):
    if len(item.stack_elevator1) == 0 and len(item.stack_elevator2) == 0:
      return "all"
    elif len(item.stack_elevator1) == 0:
      return 1
    elif len(item.stack_elevator2) == 0:
      return 2
    else:
      "None"
    
  def changeItem(item, command):
    if(len(item.stack_elevator1)>0 and len(item.stack_elevator2)>0):
      if abs(item.stack_elevator1[-1] - command[0]) <= abs(item.stack_elevator2[-1] - command[0]):
        return 1
      else:
        return 2
    else:
      if abs(item.elevator1.current_position - command[0]) <= abs(item.elevator2.current_position - command[0]):
        return 1
      else:
        return 2

  def passenger(item, command, number):
    if number == 1:
      if item.findItem(command) == item.findItem((item.stack_elevator1[-2], item.stack_elevator1[-1])):
        if command[0] >= item.stack_elevator1[-2] and command[0] <= item.stack_elevator1[-1] or command[0] <= item.stack_elevator1[-2] and command[0] >= item.stack_elevator1[-1]:
          item.stack_elevator1.insert(-1, command[0])
          item.stack_elevator1.append(command[1])
          return True
      return False
    elif number == 2:
      if item.findItem(command) == item.findItem((item.stack_elevator2[-2], item.stack_elevator2[-1])):
        if command[0] >= item.stack_elevator2[-2] and command[0] <= item.stack_elevator2[-1] or command[0] <= item.stack_elevator2[-2] and command[0] >= item.stack_elevator2[-1]:
          item.stack_elevator2.insert(-1, command[0])
          item.stack_elevator2.append(command[1])
          return True
      return False

  def optimize(item, stack):
    for i in range(len(stack) - 1, 0, -1):
      if stack[i] == stack[i-1]:
        del stack[i]

  def distribution(item):
    while(len(item.stack) > 0):
      isPassenger = False
      item.currentStep = item.stack.pop(0)
      item.setError(item.currentStep)
      if item.isError:
        return
      isEmpty = item.checkEmpty()
      if(isEmpty == "all"):
        if item.changeItem(item.currentStep) == 1:
          item.stack_elevator1.append(item.currentStep[0])
          item.stack_elevator1.append(item.currentStep[1])
        else:
          item.stack_elevator2.append(item.currentStep[0])
          item.stack_elevator2.append(item.currentStep[1])
      elif(isEmpty == 1):
        isPassenger = item.passenger(item.currentStep, 2)
        if isPassenger == False:
          item.stack_elevator1.append(item.currentStep[0])
          item.stack_elevator1.append(item.currentStep[1])
      elif(isEmpty == 2):
        isPassenger = item.passenger(item.currentStep, 1)
        if isPassenger == False:
          item.stack_elevator2.append(item.currentStep[0])
          item.stack_elevator2.append(item.currentStep[1])  
      else:
        isPassenger1 = item.passenger(item.currentStep, 2)
        isPassenger2 = item.passenger(item.currentStep, 1)
        if isPassenger1 == False and isPassenger2 == False:
          if item.changeItem(item.currentStep) == 1:
            item.stack_elevator1.append(item.currentStep[0])
            item.stack_elevator1.append(item.currentStep[1])
          else:
            item.stack_elevator2.append(item.currentStep[0])
            item.stack_elevator2.append(item.currentStep[1])
      
      item.optimize(item.stack_elevator1)
      item.optimize(item.stack_elevator2)

  def run(item):
    if item.isError:
      return
    print(item.stack_elevator1)
    print(item.stack_elevator2)
    item.elevator1.run(item.stack_elevator1)
    item.elevator2.run(item.stack_elevator2)
  
  def setError(item, command):
    if command[0] > item.floor_count or command[1] > item.floor_count or command[0] < 1 or command[1] < 1:
      print("Error!")
      item.isError = True


# Main elevator

In [23]:
class mainElevator:
  def __init__(item, start_position, numberElevator):
    item.numberElevator = numberElevator
    item.current_position = start_position
    item.stack_positions = []
    item.next_position = None
    item.isError = False
    item.isStoped = False
        
  def action(item):
    print(f"Elevator № {item.numberElevator} arrived at {item.current_position} floor")
  
  def goUp(item):
    item.current_position += 1
    print(f"Elevator {item.numberElevator} went up 1 floor")

  def goDown(item):
    item.current_position -= 1
    print(f"Elevator {item.numberElevator} went down 1 floor")

  def atRest(item):
    item.next_position = item.stack_positions.pop(0)
      

  def notMove(item):
    item.action()

  def moveUp(item):
    while item.current_position < item.next_position:
      item.goUp()
    item.notMove()

  def moveDown(item):
    while item.current_position > item.next_position:
      item.goDown()
    item.notMove()

  def run(item, stack_positions):
    item.stack_positions = stack_positions
    while(len(item.stack_positions) > 0):
      item.atRest()
      if item.current_position == item.next_position:
        item.state = 2
        item.notMove()
      elif item.current_position < item.next_position:
        item.state = 3
        item.moveUp()
      else:
        item.state = 4
        item.moveDown()
    print(f"Elevator № {item.numberElevator} is at rest")

# Tests

In [27]:
testElevator1 = remoteController(3, 7, 2)
testElevator1.initStack([(3, 7), (7, 6), (5, 8), (2, 4), (3,5), (1, 2)])
testElevator1.distribution()
testElevator1.run()

Error!


In [28]:
testElevator2 = remoteController(1, 5, 9)
testElevator2.initStack([(1, 5), (5, 6), (5, 8), (2, 4), (3,5), (1, 2)])
testElevator2.distribution()
testElevator2.run()

[1, 5, 6, 8]
[2, 3, 4, 5, 1, 2]
Elevator № 1 arrived at 1 floor
Elevator 1 went up 1 floor
Elevator 1 went up 1 floor
Elevator 1 went up 1 floor
Elevator 1 went up 1 floor
Elevator № 1 arrived at 5 floor
Elevator 1 went up 1 floor
Elevator № 1 arrived at 6 floor
Elevator 1 went up 1 floor
Elevator 1 went up 1 floor
Elevator № 1 arrived at 8 floor
Elevator № 1 is at rest
Elevator 2 went down 1 floor
Elevator 2 went down 1 floor
Elevator 2 went down 1 floor
Elevator № 2 arrived at 2 floor
Elevator 2 went up 1 floor
Elevator № 2 arrived at 3 floor
Elevator 2 went up 1 floor
Elevator № 2 arrived at 4 floor
Elevator 2 went up 1 floor
Elevator № 2 arrived at 5 floor
Elevator 2 went down 1 floor
Elevator 2 went down 1 floor
Elevator 2 went down 1 floor
Elevator 2 went down 1 floor
Elevator № 2 arrived at 1 floor
Elevator 2 went up 1 floor
Elevator № 2 arrived at 2 floor
Elevator № 2 is at rest
